<a href="https://colab.research.google.com/github/NamanJhawar-NMIMS/BigData_project/blob/main/Advanced_Big_Data_Analytics_Project_Mid_term.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Advanced Big Data Analytics - MID TERM assignment.**







Prepare necessary Installations.

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 199 kB 49.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=d4002f9a7aa22fdc5c83ec2bfdbe537037452ec6af91d4980403972f8227081b
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=adadc1abf922f84d5a6725b1adf6e776c598fd7f2f032fccb3679524ab077032
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


import all the necessary packages

In [3]:
import pyspark
import os
import wget
from zipfile import ZipFile
import time
from datetime import datetime, timedelta

from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkContext, SparkSession

from pyspark.sql.types import StringType,DateType
from pyspark.sql.functions import udf


Extract the data from the Github repository

In [4]:
# download files from repository. This contains 14 files and 2nd wget file is a correction file.
wget.download('https://github.com/NamanJhawar-NMIMS/BigData_project/raw/main/BigData_mart_data.zip', 'datafiles.zip')
# Extracting the contents from the zip file in a new directory
with ZipFile('datafiles.zip','r') as file:
  file.extractall('datafiles_dir')

time.sleep(5) #sleep added so as to make sure the autorefresh in collab has taken place to reflect the folder
wget.download('https://raw.githubusercontent.com/NamanJhawar-NMIMS/BigData_project/main/productbatch.csv','datafiles_dir/productbatch.csv')

'datafiles_dir/productbatch (1).csv'

Correct filenames and delete incorrect files

In [5]:
#After importing the data, remove old productbatch.csv and rename the new file.
os.remove('datafiles_dir/productbatch.csv')
os.rename('datafiles_dir/productbatch (1).csv','datafiles_dir/productbatch.csv')

Create the spark Context 

In [6]:
# Creating a spark object

if __name__ == "__main__":
  conf=SparkConf().setAppName('MidTermAssignment_project').setMaster('local')
  sc=SparkContext(conf=conf)

### **Resilient Distributed Dataset (RDD)**

In [90]:
# Creating a RDD directly from CSV file using TextFile; Resilient Distributed Dataset

branch_rdd=sc.textFile('datafiles_dir/branch.csv')
rewards_rdd=sc.textFile('datafiles_dir/rewards.csv')
product_rdd=sc.textFile('datafiles_dir/product.csv')

In [8]:
# Displaying the type of variable

print('Type:', type(branch_rdd))

Type: <class 'pyspark.rdd.RDD'>


In [9]:
# Count action helps to count number of rows in RDD

print('Number of rows:', branch_rdd.count())

Number of rows: 11


In [10]:
#First action displays the first row

print('first row of RDD:\n', branch_rdd.first())

first row of RDD:
 BranchID,BranchName,AddressLine1,AddressLine2,City,Country,PinCode,PhoneNumber,emailID,createdatetime,updatedatetime


In [11]:
# Take action displays the specified number of elements from an RDD

print('First 2 rows of RDD :\n', branch_rdd.take(2))

First 2 rows of RDD :
 ['BranchID,BranchName,AddressLine1,AddressLine2,City,Country,PinCode,PhoneNumber,emailID,createdatetime,updatedatetime', 'BCH0000001,Airoli,Dmart Road,"Sector 10A, Airoli",Mumbai,India,400708,9830022519,airoli@dmart.com,27-11-21 12:31,27-11-21 12:31']


In [12]:
# Top Action for fetching rows on sorted basis

print('Sorted 2 rows of RDD :\n', branch_rdd.top(2))

Sorted 2 rows of RDD :
 ['BranchID,BranchName,AddressLine1,AddressLine2,City,Country,PinCode,PhoneNumber,emailID,createdatetime,updatedatetime', 'BCH0000010,Tilak,Swagath Mall,Tilak Nagar,Bengaluru,India,560011,9960078601,tilaknagar@dmart.com,27-11-21 13:09,27-11-21 13:09']


In [13]:
# saveasTextFile is used to write on distributed storage system (HDFS or local system)

branch_rdd.saveAsTextFile('branch_rdd.csv')
print('File saved')

File saved


In [14]:
#Map transformation to split dataset and collect will display the

symbols=branch_rdd.map(lambda x:x.split(' ')[0])
print(symbols.collect())

['BranchID,BranchName,AddressLine1,AddressLine2,City,Country,PinCode,PhoneNumber,emailID,createdatetime,updatedatetime', 'BCH0000001,Airoli,Dmart', 'BCH0000002,GT', 'BCH0000003,Balanagar,Medak', 'BCH0000004,Metro', 'BCH0000005,Antrix,Dmart', 'BCH0000006,Jadavpur,Mauza', 'BCH0000007,AMR', 'BCH0000008,Kandivali,Mahavir', 'BCH0000009,Sanath,277', 'BCH0000010,Tilak,Swagath']


In [15]:
# Collect function to showcase all the values
branch_rdd.collect()

['BranchID,BranchName,AddressLine1,AddressLine2,City,Country,PinCode,PhoneNumber,emailID,createdatetime,updatedatetime',
 'BCH0000001,Airoli,Dmart Road,"Sector 10A, Airoli",Mumbai,India,400708,9830022519,airoli@dmart.com,27-11-21 12:31,27-11-21 12:31',
 'BCH0000002,GT Road,GT Road,Nehru Nagar III,Ghaziabad,India,201001,8695526985,chaudharymall@dmart.com,27-11-21 13:09,27-11-21 13:09',
 'BCH0000003,Balanagar,Medak - Hyderabad Rd,Balanagar,Hyderabad,India,500037,7005269854,balanagar@dmart.com,27-11-21 13:09,27-11-21 13:09',
 'BCH0000004,Metro Mall,Rajeswari Colony,Virugambakkam,Chennai,India,600092,7484335870,chandrametromall@dmart.com,27-11-21 13:09,27-11-21 13:09',
 'BCH0000005,Antrix,Dmart Chokdi,Nikol,Ahmedabad,India,380038,9985745007,antrixarcade@dmart.com,27-11-21 13:09,27-11-21 13:09',
 'BCH0000006,Jadavpur,Mauza Barakholla,Jadavpur,Kolkata,India,700099,9073677847,jadavpur@dmart.com,27-11-21 13:09,27-11-21 13:09',
 'BCH0000007,AMR Park,AMR Business Park,Hongasandra,Bengaluru,India

In [16]:
# take, top, first functions difference

# take returns the values in the same sequence as it was in the RDD
# top returns the values post sorting into descending order
# first returns the first value in the RDD

take_rdd = sc.parallelize([1,2222,3,4,5])
print(take_rdd.take(3))
print(take_rdd.top(4))
print(take_rdd.first())

[1, 2222, 3]
[2222, 5, 4, 3]
1


In [17]:
# extract the updatedatetime column using the Map function 
updatedatetime_list=branch_rdd.map(lambda x:x.split(',')[10])
updatedatetime_list.collect()

['updatedatetime',
 '27-11-21 12:31',
 '27-11-21 13:09',
 '27-11-21 13:09',
 '27-11-21 13:09',
 '27-11-21 13:09',
 '27-11-21 13:09',
 '27-11-21 13:09',
 '27-11-21 13:09',
 '27-11-21 13:09',
 '27-11-21 13:09']

In [83]:
# Reduce RDD
def splitFunction(x):
  x=x.split(',')[1]
  if x.isnumeric():
    return int(x)
  else:
    return 0

totalRewardsEarned=rewards_rdd.map(splitFunction)
print(totalRewardsEarned.collect())
print('Total reward earned till date:',totalRewardsEarned.reduce(lambda x,y:int(x)+int(y)))

[0, 820, 852, 1870, 1856, 8, 1365, 261, 782, 1729, 647, 1094, 346, 864, 1066, 767, 1163, 1895, 815, 1581, 766, 301, 389, 1153, 1436, 1044, 850, 1611, 1269, 192, 617, 1633, 439, 1952, 1905, 1263, 1505, 754, 1075, 1244, 769, 449, 1230, 1583, 644, 1143, 1412, 540, 696, 595, 1863, 1222, 1029, 1109, 1611, 1122, 562, 424, 903, 1998, 1665, 407, 850, 114, 162, 1637, 599, 718, 682, 33, 1708, 1934, 225, 1523, 551, 1184, 939, 172, 561, 127, 329, 1119]
Total reward earned till date: 79322


In [89]:
# Use filter in RDD
# find the row for a specific customer
input_filtered=rewards_rdd.filter(lambda row : 
                                  (row.split(",")[0]=="CUS0000160"))
print(input_filtered.count(), input_filtered.collect())

1 ['CUS0000160,820,795,29-11-21 12:00,29-11-21 12:00']


In [88]:
# use filter in RDD
filter_rdd_2 = sc.parallelize(['Rahul', 'Swati', 'Rohan', 'Shreya', 'Priya'])
print(filter_rdd_2.filter(lambda x: x.startswith('R')).collect())


['Rahul', 'Rohan']


In [20]:
# union function to join 2 rdd outputs
union_rdd_1 = union_inp.filter(lambda x: x % 2 == 0)
union_rdd_2 = union_inp.filter(lambda x: x % 3 == 0)
print(union_rdd_1.union(union_rdd_2).collect())

[2, 4, 6, 8, 6, 9]


In [ ]:
# flatmap

In [21]:
# flatmap

flatmap_rdd = sc.parallelize(["Hey there", "This is PySpark RDD Transformations"])
(flatmap_rdd.flatMap(lambda x: x.split(" ")).collect())

['Hey', 'there', 'This', 'is', 'PySpark', 'RDD', 'Transformations']

In [22]:
# paired RDD

marks = [('Rahul', 88), ('Swati', 92), ('Shreya', 83), ('Abhay', 93), ('Rohan', 78)]
sc.parallelize(marks).collect()

[('Rahul', 88), ('Swati', 92), ('Shreya', 83), ('Abhay', 93), ('Rohan', 78)]

In [ ]:
# Reduce by Key function in RDD

marks_rdd = sc.parallelize([('Rahul', 25), ('Swati', 26), ('Shreya', 22), ('Abhay', 29), ('Rohan', 22), ('Rahul', 23), ('Swati', 19), ('Shreya', 28), ('Abhay', 26), ('Rohan', 22)])
print(marks_rdd.reduceByKey(lambda x, y: x + y).collect())

In [ ]:
# sort by key 
marks_rdd = sc.parallelize([('Rahul', 25), ('Swati', 26), 
                            ('Shreya', 22), ('Abhay', 29), 
                            ('Rohan', 22), ('Rahul', 23), 
                            ('Swati', 19), ('Shreya', 28), 
                            ('Abhay', 26), ('Rohan', 22)])

print(marks_rdd.sortByKey('ascending').collect())

In [ ]:
# group by key

marks_rdd = sc.parallelize([('Rahul', 25), ('Swati', 26), 
                            ('Shreya', 22), ('Abhay', 29), ('Rohan', 22), 
                            ('Rahul', 23), ('Swati', 19), ('Shreya', 28), 
                            ('Abhay', 26), ('Rohan', 22)])
dict_rdd = marks_rdd.groupByKey().collect()
for key, value in dict_rdd:
    print(key, list(value))

In [ ]:
# count by key 

marks_rdd = sc.parallelize([('Rahul', 25), ('Swati', 26), ('Rohan', 22), 
                            ('Rahul', 23), ('Swati', 19), ('Shreya', 28), 
                            ('Abhay', 26), ('Rohan', 22)])
dict_rdd = marks_rdd.countByKey().items()
for key, value in dict_rdd:
    print(key, value)

## **Dataframes in pyspark**

In [ ]:
session=SparkSession.builder.appName("NewApplication_midterm_dataframes").master('local').getOrCreate()

In [ ]:
# dynamically load all the csv files into pyspark object
# using this we have dynamically created objects. 
path='datafiles_dir'
dataframe_file_dict={} # The key of this dictionary will act as an object to a pyspark dataframe
for file in os.listdir(path):
  dataframe_file_dict[file]=session.read.csv(f'{path}/{file}',inferSchema=True,header=True)
  #object=session.read.csv(filename)


In [ ]:
# confirm the datatype of the dynamically created object
type(dataframe_file_dict['branch.csv'])

In [ ]:
# Show function is used to print the dataframe on the screen
dataframe_file_dict['branch.csv'].show()

In [ ]:
# print schema is used to show the data type 
dataframe_file_dict['branch.csv'].printSchema()

In [ ]:
  # columns function is used to print the columns that are there for a particular dataframe
  dataframe_file_dict['branch.csv'].columns

In [ ]:
  # dtype function is used to print a list of tuples the columns and their corresponding type
  # that are there for a particular dataframe
  dataframe_file_dict['branch.csv'].dtypes

## Convert all the objects into tempView tables 

In [ ]:
# create tempViews for all the dataframes and name them 'objectName_tdb'
for dataframe_object in list(dataframe_file_dict.keys()):
  dataframe_file_dict[dataframe_object].createOrReplaceTempView(f"{dataframe_object.split('.')[0]}_tdb")


In [ ]:
# check if the tempviews were created sucessfully
from pyspark import SparkContext 
from pyspark.sql import SQLContext 
sqlc = SQLContext(sc)

for tempView in sqlc.tableNames():
  print(tempView)

## Using these tables, let's create queries

Query 1: As an Inventory manager you want to look at which stock is closest to get over so that you can plan to order it immediately. Find the 
stock which is there the least.
Also find the supplier and their contact info.

In [ ]:
session.sql('''select p.productid,p.productname, Available, Quantity, Price,p.SupplierID,s.SupplierName,s.PhoneNumber 
from product_tdb p, supplier_tdb s 
where p.SupplierID=s.SupplierID and p.Available='N' 
and p.Quantity=(select min(Quantity) from product_tdb) 
order by s.supplierid''').show()


Query 2: Which is the highest selling product in dmart.

In [ ]:
session.sql('''
Select P.ProductName, sum(O.quantity) from orderitem_tdb O inner join product_tdb P on P.productid=O.productid group by P.ProductName order 
by 2 desc limit 1;
''').show()

**Query 3**: Find how many items the customers have shopped with us

In [ ]:
session.sql('''
select customerid, count(customerid) 
from order_tdb 
group by CustomerID 
having count(customerid) >2 
order by 2 desc
'''.replace('\n',' ')).show()

**Query 4**: What is the average bill of a customer?


In [ ]:
session.sql("""
SELECT ROUND(AVG(ordertotal), 2) as Customer_Average_Bill FROM order_tdb;
""").show()

**Query 5**: Top 5 Spenders

In [ ]:
session.sql("""
SELECT C.Customerid,CONCAT(C.FirstName, ' ',C.LastName) as FullName,
 C.MobileNumber,ROUND(AVG(O.ordertotal), 2) as average_spend
FROM order_tdb O, customer_tdb C 
WHERE O.customerid = C.customerid 
GROUP BY C.customerid, CONCAT(C.FirstName, ' ',C.LastName),C.MobileNumber 
ORDER BY AVG(O.OrderTotal) DESC 
LIMIT 5
""").show()

**Query 6**: As an inventory manager you want to understand how many products do we have for each product category.


In [ ]:
session.sql("""
Select p.Categoryid, pc.categorytype, COUNT(p.categoryID) 
FROM product_tdb p, product_category_tdb pc 
WHERE p.categoryID=pc.CategoryID 
GROUP BY p.CategoryID, pc.CategoryType 
ORDER BY COUNT(p.CategoryID) DESC;
""").show()

**Query 7**: Products that are about to expire in the next 6 months and luxury


In [ ]:
session.sql("""
SELECT P.productname, B.BrandName, B.BrandType, PB.expiryDate 
FROM productbatch_tdb PB, Product_tdb P, brand_tdb B 
WHERE PB.ProductID =P.Productid 
 AND P.Brandid = B.Brandid 
 AND PB.expiryDate > (current_date) 
 AND PB.expiryDate < DATE_ADD((current_date), 180);
""").show()

**Query 8**: As the manager you want to understand the generations from which your employees come. This would help you bring HR related 
changes in the future.

In [ ]:
def str_to_date(dob):
  return datetime.strptime(dob, '%d-%m-%y')

session.udf.register("str_to_date_udf", str_to_date,DateType())


session.sql("""
SELECT CONCAT(Firstname, ' ', lastname) as FullName, str_to_date_udf(dob) as DOB ,
 CASE WHEN YEAR(str_to_date_udf(dob)) BETWEEN 1946 AND 1964 THEN 'Baby Boomers' 
 WHEN YEAR(str_to_date_udf(dob)) BETWEEN 1965 AND 1980 THEN 'Gen X' 
 WHEN YEAR(str_to_date_udf(dob)) BETWEEN 1981 AND 1996 THEN 'Millennial' 
 ELSE 'Gen Z' END as Generation
FROM employee_tdb""").show()

**Query 9**: Find the products that have been sold more than 50 quantities

In [ ]:
session.sql("""
SELECT P.productid, P.ProductName, SUM(OI.Quantity) AS QuantitySold 
FROM orderitem_tdb OI, product_tdb P 
WHERE P.productid = OI.productid 
GROUP BY P.productid,P.ProductName 
HAVING QuantitySold > 50 
order by QuantitySold desc
""").show()

**Query 10**: How many brands are there for each brand type?


In [ ]:
session.sql("""
SELECT brandtype, COUNT(*) as  quantity 
FROM Brand_tdb 
GROUP BY brandtype 
ORDER BY 2
""").show()

**Query 11**: No of cities in which Dmart has branches


In [ ]:
session.sql("""
SELECT COUNT(DISTINCT city) as Number_of_Cities FROM branch_tdb
""").show()

**Query 12**: What are the total payments done via different modes? The transactions need to be successful.


In [ ]:
session.sql("""
select sum(amount), paymentmode from transaction_tdb where TransactionStatus='Success' group by PaymentMode order by 1 desc;
""").show()

**Query 13**:What are the demographics of the customer base?


In [ ]:
session.sql("""
select count(gender), gender from customer_tdb group by gender;

""").show()

**Query 14**:The store wants to update all it’s Security Guards regarding an upcoming festival. We need their phone numbers to send them an sms regarding the same.


In [ ]:
session.sql("""
select concat(e.firstname,e.lastname), e.mobilenumber, s.SkillName from employee_tdb e, skill_tdb s where e.skill_code=s.Skill_code and SkillName='Security';
""").show()

**Query 15**:Select the Products that are not in the Luxury segment

In [ ]:
session.sql("""
SELECT
	P.ProductName, B.BrandName, B.BrandType
FROM
	brand_tdb B,
	product_tdb P
WHERE
	B.BrandID = P.BrandID
    	AND B.BrandType != 'Luxury';

""").show()

**Query 16**:Customer who has earned the most reward points


In [ ]:
session.sql("""
SELECT CONCAT(C.Firstname, ' ', C.lastname),(R.TotalRewardPointsEarned + R.TotalRewardPointRedeemed) as totalPoints 
FROM customer_tdb C, rewards_tdb R 
WHERE C.customerid = R.customerid 
ORDER BY 2 DESC LIMIT 1""").show()

**Query 17**:The business wants to run a campaign to promote their Rewards program with their existing customers who are currently not part of the same.
Find the list of Names and Mobile numbers so that you can run that campaign.alter


In [ ]:
session.sql("""
SELECT 
  S.Suppliername, S.PhoneNumber 
FROM 
  supplier_tdb S, Supplier_branch_relation_tdb SBR 
WHERE 
  SBR.supplierid = S.supplierid AND SBR.branchid 
IN 
  (SELECT branchid FROM branch_tdb WHERE city = 'Mumbai');
""").show()

**Query 18**: Top 3 brands that have generated the highest revenue


In [ ]:
session.sql("""
SELECT 
  B.brandname,B.brandtype, ROUND(SUM(OI.subtotal + OI.tax - OI.ItemDiscount),2)
FROM 
  Brand_tdb B, Product_tdb P, Orderitem_tdb OI 
WHERE 
  B.brandid = P.brandid AND P.Productid = OI.productid
GROUP BY 
  B.brandname, B.brandtype
ORDER BY
  3 DESC 
LIMIT 3;
""").show()

**Query 19**:Due to rising cases in the 4th wave, the Dmart stores in Mumbai are going to be closed for a few days. We need to inform the suppliers to defer the standing orders.Find the names and mobile number of the suppliers so that you can contact them.


In [ ]:
session.sql("""
SELECT 
  S.Suppliername, S.PhoneNumber 
FROM 
  supplier_tdb S, Supplier_branch_relation_tdb SBR 
WHERE SBR.supplierid = S.supplierid 
AND 
  SBR.branchid 
IN 
  (SELECT branchid FROM branch_tdb WHERE city = 'Mumbai');
""").show()

**Query 20**:Find the names and contact info of all employees who are managers and based in Mumbai as the company wants to provide instructions regarding an upcoming mega sale in the Mumbai stores.


In [ ]:
session.sql("""
SELECT 
  firstname, lastname, MobileNumber, B.city 
FROM 
  employee_tdb E,Branch_tdb B 
WHERE 
  B.branchid = E.branchid AND B.city = 'Mumbai' AND E.skill_code 
IN 
  (SELECT skill_code FROM skill_tdb WHERE skillname LIKE '%Manager%')
""").show()

## Column Transformations

In [ ]:
dataframe_file_dict['product.csv'].show(10)

In [ ]:
# Print the quantity of products available
dataframe_file_dict['product.csv'].select('ProductName','Quantity').show(5)

In [ ]:
# create a User defined function and use this in SQL and column transformation to generate the fullname of the employee

def fullName(firstname,lastname):
  return firstname+" "+lastname

session.udf.register("fullName_udf", fullName,StringType())

fullName_col_udf=udf(fullName,StringType())




In [ ]:
result=dataframe_file_dict['employee.csv'].withColumn('fullName',fullName_col_udf("FirstName","LastName"))
result.select('EmployeeID','FirstName','lastname','Fullname').show(10)

In [ ]:
session.sql('select fullname_udf(firstname,lastname) FROM employee_tdb').show(5)

In [ ]:
# Ban products using the utf column transformation
ban_product='BRD0000006'
def ban_products(BrandID):
  if BrandID==ban_product:
    return 'Y'
  else:
    return 'N'
  
ban_products_udf=udf(ban_products,StringType())

result=dataframe_file_dict['product.csv'].withColumn('ban_status',ban_products_udf(dataframe_file_dict['product.csv'].BrandID))
result.select('BrandID','ProductID','ProductName','Available','ban_status').show(10)

In [ ]:
# return all those products which have been banned
result.select('BrandID','ProductID','ProductName','Available','ban_status').where(result.ban_status=='Y').show()